In [3]:
# Processing and Displaying run results
import os
import pandas as pd

In [4]:
# Define a function to extract information from the file content
def extract_info_from_file(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    total_time = None
    final_accuracy = None
    for line in reversed(lines):
        if 'Total training time:' in line:
            total_time = float(line.strip().split(': ')[1].split(' ')[0])
        if 'accuracy:' in line:
            final_accuracy = float(line.strip().split(': ')[1].replace('%', ''))
            break  # Stop after the final accuracy is found as it is the first instance from the end

    return total_time, final_accuracy

# Function to parse file name
def parse_filename(file_name):
    parts = file_name.split('-')
    if len(parts) == 5:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3].split('.')[0]  # Remove file extension, prob need to add run number here
    else:
        optimizer, model, dataset, augmentation = None, None, None, None

    return optimizer, model, dataset, augmentation

In [5]:
# Specify the directory
folder_path = 'logs'  # Change 'folder_name' to the name of your folder

# DataFrame to store the results
results_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Total Time (s)', 'Final Accuracy (%)'])

results_df

,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%)


In [6]:
# Process each file in the directory
for file in os.listdir(folder_path):
    if file.endswith('.out'):  # Check if the file is a .out file
        file_path = os.path.join(folder_path, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        optimizer, model, dataset, augmentation = parse_filename(file)
        
        # Append the results to the DataFrame
        results_df = results_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time
        }, ignore_index=True)

results_df['Total Time (min)'] = results_df['Total Time (s)'] / 60

results_df.head()

,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%),Total Time (min)


In [7]:
grouped_df = results_df.groupby(['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation']).agg({
    'Final Accuracy (%)': ['mean', 'std'],
    'Total Time (min)': ['mean', 'std']
}).reset_index()

# Rename the columns for clarity
grouped_df.columns = [
    'Optimizer', 
    'Model Architecture', 
    'Dataset', 
    'Data Augmentation', 
    'Final Accuracy Mean (%)', 
    'Final Accuracy Std (%)',
    'Total Time Mean (min)', 
    'Total Time Std (min)' 
]

grouped_df

,Optimizer,Model Architecture,Dataset,Data Augmentation,Final Accuracy Mean (%),Final Accuracy Std (%),Total Time Mean (min),Total Time Std (min)


In [8]:
df_sorted = grouped_df.sort_values(by=['Dataset', 'Data Augmentation'])

df_sorted

,Optimizer,Model Architecture,Dataset,Data Augmentation,Final Accuracy Mean (%),Final Accuracy Std (%),Total Time Mean (min),Total Time Std (min)


In [9]:
from tabulate import tabulate

In [10]:
# Define the custom order for Data Augmentation
augmentation_order = ["basicaug", "cutout", "autoaug", "randaug"]
optimizer_order = ["sgd", "nonaccelgamsgd", "gamsgd"]

# Create a categorical type for Data Augmentation with the specified order
grouped_df["Data Augmentation"] = pd.Categorical(grouped_df["Data Augmentation"], categories=augmentation_order, ordered=True)

# Filter and sort DataFrame to match the desired format
df_sorted = grouped_df.sort_values(by=["Dataset", "Data Augmentation", "Optimizer"])
df_sorted = df_sorted.loc[df_sorted["Data Augmentation"] != "randaug"]
df_cifar10 = df_sorted[df_sorted["Dataset"] == "cifar10"]
df_cifar100 = df_sorted[df_sorted["Dataset"] == "cifar100"]

# Prepare the CIFAR-10 and CIFAR-100 parts of the table
table_cifar10 = df_cifar10.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()
table_cifar100 = df_cifar100.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()

table_cifar10 = table_cifar10[["Data Augmentation"] + optimizer_order]

# Add a new column to distinguish between CIFAR-10 and CIFAR-100
table_cifar10.insert(0, 'Dataset', 'CIFAR-10')
table_cifar100.insert(0, 'Dataset', 'CIFAR-100')

# Combine the tables
table = pd.concat([table_cifar10, table_cifar100], axis=0)

# Print the table
print(tabulate(table, headers='keys', tablefmt='pipe'))

KeyError: "['sgd', 'nonaccelgamsgd', 'gamsgd'] not in index"

### Gradient Norm Only Minimization (GNOM)

#### GNOM learning rate testing (20 epochs):

In [11]:
# Specify the directory
folder_path_gnom = 'logs/GNOM-Tests'  


In [12]:
def extract_epoch_time(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    epoch_time = 0
    count = 0
    for line in lines:
        if 'Epoch' and 'time:' in line:
            count += 1
            epoch_time += float(line.strip().split(': ')[1].split(' ')[0])
    return (epoch_time / count)

def extract_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    epochs = 20
    if len(parts) >= 6:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
    else:
        optimizer, model, dataset, augmentation, learning_rate, batch_size = None, None, None, None, None, None
    
    if len(parts) == 7:
        learning_rate = parts[5]
        batch_size = parts[6].split('.')[0] 
        if batch_size == 'full':
            batch_size = 128
            epochs = 200
    else:
        learning_rate = parts[5][:-4]
        batch_size = 128
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs

In [13]:
gnom_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom):
    if '-lr-' in file:  
        file_path = os.path.join(folder_path_gnom, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        epoch_time = extract_epoch_time(file_path)
        optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = extract_test_info(file_path)
        
        gnom_df = gnom_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Learning Rate': float(learning_rate),
            'Batch Size': int(batch_size),
            'Epochs': epochs,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time,
            'Avg Epoch Time (s)': epoch_time
        }, ignore_index=True)

gnom_df['Total Time (min)'] = gnom_df['Total Time (s)'] / 60

gnom_df_sorted = gnom_df.sort_values(by=['Batch Size', 'Learning Rate'])

gnom_df_sorted

/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_958/9759538.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gnom_df = gnom_df._append({


,Optimizer,Model Architecture,Dataset,Data Augmentation,Learning Rate,Batch Size,Epochs,Final Accuracy (%),Total Time (s),Avg Epoch Time (s),Total Time (min)
7,gnomsgd,resnet18,cifar10,basicaug,0.010,128,20,10.30,2390.612709,227.676276,39.843545
10,gnomsgd,resnet18,cifar10,basicaug,0.050,128,20,18.06,2398.976304,228.472647,39.982938
14,gnomsgd,resnet18,cifar10,basicaug,0.050,128,200,10.00,23864.758999,237.459076,397.745983
1,gnomsgd,resnet18,cifar10,basicaug,0.100,128,20,10.00,2394.460447,228.042485,39.907674
2,gnomsgd,resnet18,cifar10,basicaug,0.200,128,20,10.00,2384.751838,227.118328,39.745864
6,gnomsgd,resnet18,cifar10,basicaug,0.400,128,20,10.00,2393.539392,227.954680,39.892323
11,gnomsgd,resnet18,cifar10,basicaug,0.010,256,20,17.04,2241.874085,213.510471,37.364568
5,gnomsgd,resnet18,cifar10,basicaug,0.050,256,20,10.71,2237.419777,213.086812,37.290330
8,gnomsgd,resnet18,cifar10,basicaug,0.100,256,20,14.33,2247.514600,214.047492,37.458577
12,gnomsgd,resnet18,cifar10,basicaug,0.200,256,20,9.97,2252.608104,214.532993,37.543468


#### GNOM Gradient Norm and Hessian Eigenvalue testing:

In [27]:
folder_path_gnom_ev = 'logs/GNOM-GradNorm-Eigenvalues'  

def extract_ev_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    if len(parts) == 8:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
        learning_rate = parts[5]
        batch_size = parts[6]
        epochs = parts[7][:-4]
    else:
        optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = None, None, None, None, None, None, None
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs

def extract_grad_info(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    grad_norm = None
    top_ev = None
    for line in reversed(lines):
        if 'Norm of the Gradient:' in line:
            grad_norm = float(line.strip().split(': ')[1])
        if 'The top Hessian eigenvalue of this model is' in line:
            top_ev = float(line.strip().split(' ')[-1])
            break

    return grad_norm, top_ev

In [30]:
gnom_ev_df = pd.DataFrame(columns=['Optimizer', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom_ev):
    file_path = os.path.join(folder_path_gnom_ev, file)
    total_time, final_accuracy = extract_info_from_file(file_path)
    epoch_time = extract_epoch_time(file_path)
    optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = extract_ev_test_info(file_path)
    grad_norm, top_ev = extract_grad_info(file_path)
    
    gnom_ev_df = gnom_ev_df._append({
        'Optimizer': optimizer,
        'Learning Rate': float(learning_rate),
        'Batch Size': int(batch_size),
        'Epochs': epochs,
        'Final Accuracy (%)': final_accuracy,
        'Total Time (s)': total_time,
        'Avg Epoch Time (s)': epoch_time,
        'Gradient Norm': grad_norm,
        'Largest Eigenvalue of Hessian': top_ev
    }, ignore_index=True)

gnom_ev_df['Total Time (min)'] = gnom_ev_df['Total Time (s)'] / 60
gnom_ev_df['Avg Epoch Time (min)'] = gnom_ev_df['Avg Epoch Time (s)'] / 60
gnom_ev_df.drop(columns=['Total Time (s)', 'Avg Epoch Time (s)'], inplace=True)

gnom_ev_df_sorted = gnom_ev_df.sort_values(by=['Optimizer','Batch Size'])

print("All of the below runs were done with ResNet18, CIFAR-10, and basic data augmentations")
gnom_ev_df_sorted


All of the below runs were done with ResNet18, CIFAR-10, and basic data augmentations


/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_958/910117920.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gnom_ev_df = gnom_ev_df._append({


,Optimizer,Learning Rate,Batch Size,Epochs,Final Accuracy (%),Gradient Norm,Largest Eigenvalue of Hessian,Total Time (min),Avg Epoch Time (min)
2,gamsgd,0.10,128,50,94.77,0.915585,34.1805,83.035610,3.256290
4,gnomsgd,0.05,128,50,10.00,0.071134,0.1001,99.177627,3.889305
0,gnomsgd,0.01,256,50,10.66,0.055074,0.1162,93.000017,3.647047
1,gnomsgd,0.01,512,50,14.95,0.119978,0.6439,89.388052,3.505400
5,nonaccelgamsgd,0.10,128,50,94.42,1.223947,85.2085,197.052924,7.727553
3,sgd,0.10,128,50,94.25,0.575474,157.5169,20.854020,0.817796
